# The Writer Agent: Synthesizing Information in Multi-Agent Systems

*By Rany ElHousieny*

*Last updated: May 17, 2025*

## Introduction

In this notebook, we'll explore the Writer agent, a crucial component of multi-agent AI systems. The Writer agent synthesizes information from the Researcher and feedback from the Critic to create coherent, well-structured responses. We'll implement a Writer agent using LangChain and GPT-4.1, and demonstrate how it enhances the quality of AI-generated content.

## Setting Up the Environment

First, let's install the necessary packages and set up our environment:

In [1]:
# Install required packages
!pip install langchain-openai python-dotenv openai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
# Import necessary libraries
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()  # This loads the OPENAI_API_KEY from your .env file

True

In [3]:
# Import LangChain components
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Verify that we can connect to the OpenAI API
llm = ChatOpenAI(
    temperature=0.7, 
    model_name="gpt-4.1",  # Using GPT-4.1 as per user preference
    api_key=os.environ.get("OPENAI_API_KEY")
)

# Test the connection with a simple query
llm.invoke("Hello, are you ready to learn about the Writer agent?")

AIMessage(content='Hello! Yes, I’m always ready to learn. Please tell me about the Writer agent—what is it, and what would you like me to know or do regarding it?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 19, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_3b5c2e113e', 'finish_reason': 'stop', 'logprobs': None}, id='run--69ce9017-760e-4bab-a726-29b5fb967834-0', usage_metadata={'input_tokens': 19, 'output_tokens': 36, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## The Role of the Writer Agent

The Writer agent serves as the final synthesizer within multi-agent systems. While other agents (like Researchers or Critics) focus on generating or evaluating content, the Writer transforms these inputs into a coherent, user-friendly final product. This synthesis helps:

1. **Organize information** in a logical, easy-to-understand structure
2. **Integrate feedback** from the Critic to address gaps and inconsistencies
3. **Balance different perspectives** to ensure objectivity
4. **Present complex information** in a clear, engaging style
5. **Create a unified voice** despite drawing from multiple sources

Let's define the system prompts for all three agents in our multi-agent system:

In [4]:
# Define the Researcher Agent's system prompt
RESEARCHER_SYSTEM_PROMPT = """
You are a Researcher Agent, part of a collaborative research assistant system. Your role is to gather and provide 
accurate, relevant information on any topic or question presented to you.

Your responsibilities include:
1. Analyzing research questions to understand what information is being requested
2. Providing comprehensive, well-structured responses based on your knowledge
3. Highlighting key points and important information
4. Being honest about the limitations of your knowledge
5. Maintaining objectivity and avoiding bias in your responses

Format your responses in a clear, organized manner with sections, bullet points, or numbered lists as appropriate.
Always cite your sources of information when possible.

Remember, your goal is to provide the most helpful, accurate information possible to assist in the research process.
"""

# Define the Critic Agent's system prompt
CRITIC_SYSTEM_PROMPT = """
You are a Critic Agent, part of a collaborative research assistant system. Your role is to evaluate 
and challenge information provided by the Researcher Agent to ensure accuracy, completeness, and objectivity.

Your responsibilities include:
1. Analyzing research findings for accuracy, completeness, and potential biases
2. Identifying gaps in the information or logical inconsistencies
3. Asking important questions that might have been overlooked
4. Suggesting improvements or alternative perspectives
5. Ensuring that the final information is balanced and well-rounded

Be constructive in your criticism. Your goal is not to dismiss the researcher's work, but to strengthen it.
Format your feedback in a clear, organized manner, highlighting specific points that need attention.

Remember, your ultimate goal is to ensure that the final research output is of the highest quality possible.
"""

# Define the Writer Agent's system prompt
WRITER_SYSTEM_PROMPT = """
You are a Writer Agent, part of a collaborative research assistant system. Your role is to synthesize 
information from the Researcher Agent and feedback from the Critic Agent into a coherent, well-written response.

Your responsibilities include:
1. Analyzing the information provided by the researcher and the feedback from the critic
2. Organizing the information in a logical, easy-to-understand structure
3. Presenting the information in a clear, engaging writing style
4. Balancing different perspectives and ensuring objectivity
5. Creating a final response that is comprehensive, accurate, and well-written

Format your response in a clear, organized manner with appropriate headings, paragraphs, and bullet points.
Use simple language to explain complex concepts, and provide examples where helpful.

Remember, your goal is to create a final response that effectively communicates the information to the user.
"""

## Implementing the Multi-Agent System

Now, let's implement all three agents in our multi-agent system, with a focus on the Writer agent:

In [5]:
def researcher_agent(messages):
    """Researcher agent that analyzes questions and provides informative responses."""
    # Add the system prompt to guide the researcher agent's behavior
    researcher_messages = [SystemMessage(content=RESEARCHER_SYSTEM_PROMPT)] + messages
    
    # Create a ChatOpenAI instance with appropriate parameters
    llm = ChatOpenAI(
        temperature=0.5,  # Lower temperature for more factual responses
        model_name="gpt-4.1",  # Using GPT-4.1 as per user preference
        api_key=os.environ.get("OPENAI_API_KEY")
    )
    
    # Get a response from the language model
    response = llm.invoke(researcher_messages)
    
    # Print a preview of the response for monitoring
    print("\n--- Researcher Agent Response ---")
    print(f"{response.content[:300]}...\n")
    
    return response

In [6]:
def critic_agent(messages):
    """Critic agent that evaluates and challenges the researcher's findings."""
    # Add the system prompt to guide the critic agent's behavior
    critic_messages = [SystemMessage(content=CRITIC_SYSTEM_PROMPT)] + messages
    
    # Create a ChatOpenAI instance with appropriate parameters
    llm = ChatOpenAI(
        temperature=0.7,  # Slightly higher temperature for more creative criticism
        model_name="gpt-4.1",  # Using GPT-4.1 as per user preference
        api_key=os.environ.get("OPENAI_API_KEY")
    )
    
    # Get a response from the language model
    response = llm.invoke(critic_messages)
    
    # Print a preview of the response for monitoring
    print("\n--- Critic Agent Response ---")
    print(f"{response.content[:300]}...\n")
    
    return response

In [7]:
def writer_agent(messages):
    """Writer agent that synthesizes information from the researcher and critic."""
    # Add the system prompt to guide the writer agent's behavior
    writer_messages = [SystemMessage(content=WRITER_SYSTEM_PROMPT)] + messages
    
    # Create a ChatOpenAI instance with appropriate parameters
    llm = ChatOpenAI(
        temperature=0.6,  # Balanced temperature for creativity and accuracy
        model_name="gpt-4.1",  # Using GPT-4.1 as per user preference
        api_key=os.environ.get("OPENAI_API_KEY")
    )
    
    # Get a response from the language model
    response = llm.invoke(writer_messages)
    
    # Print a preview of the response for monitoring
    print("\n--- Writer Agent Response ---")
    print(f"{response.content[:300]}...\n")
    
    return response

### Code Explanation: Writer Agent

Let's break down each line of the `writer_agent` function to understand how it works:

```python
def writer_agent(messages):
```
- This defines a function named `writer_agent` that takes a list of `messages` as input. These messages will include the original query, the Researcher's response, and the Critic's feedback.

```python
writer_messages = [SystemMessage(content=WRITER_SYSTEM_PROMPT)] + messages
```
- This creates a new list called `writer_messages` by prepending the Writer's system prompt to the existing messages.
- The `SystemMessage` class is used to format the prompt as a system message, which tells the language model what role to assume.
- This is crucial for establishing the Writer's role and responsibilities before it processes the other messages.

```python
llm = ChatOpenAI(
    temperature=0.6,  # Balanced temperature for creativity and accuracy
    model_name="gpt-4.1",  # Using GPT-4.1 for advanced capabilities
    api_key=os.environ.get("OPENAI_API_KEY")
)
```
- This creates a `ChatOpenAI` instance, which is the interface to the OpenAI API.
- The `temperature` parameter is set to 0.6, which is a middle ground between the fact-focused Researcher (0.5) and the more creative Critic (0.7). This balanced setting allows the Writer to be creative in its presentation while maintaining accuracy.
- The `model_name` parameter specifies which language model to use. We're using GPT-4.1 for its advanced capabilities in understanding complex information and generating well-structured content.
- The `api_key` parameter is set to the OpenAI API key retrieved from environment variables, which is a secure way to handle API keys.

```python
response = llm.invoke(writer_messages)
```
- This sends the prepared messages to the language model using the `invoke` method and gets a response.
- The response will contain the Writer's synthesis of the information provided by the Researcher and the feedback provided by the Critic.

```python
print("\n--- Writer Agent Response ---")
print(f"{response.content[:300]}...\n")
```
- These lines print a preview of the Writer's response for monitoring purposes.
- Only the first 300 characters are shown to avoid cluttering the output.

```python
return response
```
- This returns the full response from the Writer agent, which can then be presented to the user as the final output of the multi-agent system.

## Creating a Complete Multi-Agent Pipeline

Now, let's create a complete sequential pipeline that combines all three agents:

In [8]:
def run_multi_agent_system(question):
    """Run the multi-agent system with the given question."""
    print(f"\n\n{'='*50}")
    print(f"QUERY: {question}")
    print(f"{'='*50}\n")
    
    # Initialize the messages with the question
    messages = [HumanMessage(content=question)]
    
    # Run the researcher agent
    researcher_response = researcher_agent(messages)
    messages.append(researcher_response)
    
    # Run the critic agent
    critic_response = critic_agent(messages)
    messages.append(critic_response)
    
    # Run the writer agent
    writer_response = writer_agent(messages)
    messages.append(writer_response)
    
    return messages

### Multi-Agent Flow Diagram

The following Mermaid diagram illustrates the flow of information through our multi-agent system:

```mermaid
flowchart TB
    User([User]) --> |Question| Q[User Query]
    
    subgraph Multi-Agent System
        Q --> Researcher
        Researcher --> |Initial Response| Critic
        Q --> |Original Question| Critic
        Critic --> |Evaluation & Feedback| Writer
        Researcher --> |Initial Response| Writer
        Q --> |Original Question| Writer
        Writer --> FinalResponse[Final Response]
    end
    
    FinalResponse --> User
    
    subgraph Researcher Agent
        Researcher --> |GPT-4.1| R1[Research & Information Gathering]
        R1 --> R2[Structured Response Generation]
    end
    
    subgraph Critic Agent
        Critic --> |GPT-4.1| C1[Analysis of Research]
        C1 --> C2[Gap Identification]
        C2 --> C3[Bias Detection]
        C3 --> C4[Alternative Perspectives]
        C4 --> C5[Constructive Feedback]
    end
    
    subgraph Writer Agent
        Writer --> |GPT-4.1| W1[Analysis of Inputs]
        W1 --> W2[Feedback Integration]
        W2 --> W3[Content Organization]
        W3 --> W4[Style Enhancement]
        W4 --> W5[Final Synthesis]
    end
    
    classDef agent fill:#f9f,stroke:#333,stroke-width:2px
    classDef process fill:#bbf,stroke:#333,stroke-width:1px
    classDef data fill:#dfd,stroke:#333,stroke-width:1px
    
    class Researcher,Critic,Writer agent
    class R1,R2,C1,C2,C3,C4,C5,W1,W2,W3,W4,W5 process
    class Q,FinalResponse data
```

This diagram shows:

1. **Information Flow**: How the user's query moves through the system, being processed by each agent in sequence
2. **Agent Responsibilities**: The specific tasks performed by each agent
3. **Writer's Role**: The detailed synthesis process performed by the Writer agent
4. **Input Integration**: How the Writer receives inputs from both the Researcher and Critic

The Writer agent serves as the final step in this workflow, receiving the original question, the Researcher's information, and the Critic's feedback. It synthesizes these inputs into a coherent, well-structured response that is returned to the user.

## Testing the Multi-Agent System

Let's test our multi-agent system with a research question about artificial intelligence ethics:

In [9]:
# Run the multi-agent system with a research question
result = run_multi_agent_system("What are the ethical considerations of using AI in healthcare?")



QUERY: What are the ethical considerations of using AI in healthcare?


--- Researcher Agent Response ---
**Ethical Considerations of Using AI in Healthcare**

The integration of Artificial Intelligence (AI) into healthcare presents significant opportunities but also raises important ethical concerns. Below are key ethical considerations, organized by major themes:

---

### 1. **Privacy and Data Securi...


--- Critic Agent Response ---
**Critique of the Researcher’s Response:**

Your answer provides a thorough overview of the main ethical considerations regarding AI in healthcare and is generally well-organized. However, there are areas that could be strengthened for completeness, nuance, and clarity:

---

### 1. **Accuracy and C...


--- Writer Agent Response ---
Ethical Considerations of Using AI in Healthcare

The use of artificial intelligence (AI) in healthcare offers great promise, from improving diagnoses to expanding access to care. However, it also raises important ethica

In [10]:
# Display the full conversation
print("\n\nFull conversation:")
for i, message in enumerate(result):
    if isinstance(message, HumanMessage):
        print(f"\nHuman: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"\nAI: {message.content}")



Full conversation:

Human: What are the ethical considerations of using AI in healthcare?

AI: **Ethical Considerations of Using AI in Healthcare**

The integration of Artificial Intelligence (AI) into healthcare presents significant opportunities but also raises important ethical concerns. Below are key ethical considerations, organized by major themes:

---

### 1. **Privacy and Data Security**
- **Patient Confidentiality:** AI systems require large datasets, often containing sensitive personal health information. Ensuring data privacy and compliance with regulations (e.g., HIPAA, GDPR) is crucial.
- **Data Breaches:** Increased digitalization raises the risk of unauthorized access and data breaches.

---

### 2. **Bias and Fairness**
- **Algorithmic Bias:** AI systems can perpetuate or amplify biases present in training data, leading to unequal treatment or misdiagnosis, especially among minority or underrepresented groups.
- **Equitable Access:** Disparities in access to AI-drive

## Analyzing the Writer's Response

Let's analyze how the Writer agent synthesizes the Researcher's information and the Critic's feedback:

In [11]:
# Display just the final response from the Writer agent
print("\n\nFINAL RESPONSE FROM WRITER:")
print(result[-1].content)



FINAL RESPONSE FROM WRITER:
Ethical Considerations of Using AI in Healthcare

The use of artificial intelligence (AI) in healthcare offers great promise, from improving diagnoses to expanding access to care. However, it also raises important ethical questions that must be thoughtfully addressed. Below, we organize the main ethical considerations, balance different perspectives, and provide concrete examples to clarify these complex issues.

---

### 1. Privacy, Data Security, and Ownership

- **Patient Confidentiality:** AI needs vast amounts of patient data, often including sensitive health information. Protecting this data from breaches and misuse is crucial.
- **Data Ownership:** There is ongoing debate about who owns health data—patients, hospitals, or companies developing AI tools. Clear policies are needed to safeguard patient rights.
- **Global Differences:** Privacy expectations and regulations (like HIPAA in the US or GDPR in Europe) vary by country, so ethical standards mus

### Key Aspects of the Writer's Synthesis

In the Writer's response, we can observe several key aspects of effective synthesis:

1. **Structural Organization**: The Writer organizes information in a logical flow, with clear headings and a progression from foundational concepts to specific applications and considerations.

2. **Technical Accessibility**: Complex concepts are explained in accessible language, often with analogies or examples to aid understanding.

3. **Feedback Integration**: The Writer addresses gaps and issues identified by the Critic, such as incorporating alternative perspectives or balancing different aspects of the topic.

4. **Balanced Presentation**: Different viewpoints are presented in a balanced way, acknowledging both benefits and challenges.

5. **Engaging Style**: The Writer uses a clear, engaging writing style that maintains reader interest while conveying complex information.

## Comparing Agent Outputs

Let's compare the outputs from each agent to see how the information evolves through the multi-agent system:

In [12]:
# Extract the responses from each agent
researcher_output = result[1].content
critic_output = result[2].content
writer_output = result[3].content

# Compare the length of each response
print(f"Researcher output length: {len(researcher_output)} characters")
print(f"Critic output length: {len(critic_output)} characters")
print(f"Writer output length: {len(writer_output)} characters")

Researcher output length: 3834 characters
Critic output length: 3526 characters
Writer output length: 5231 characters


## The Value of the Writer in the Complete Pipeline

The Writer agent adds significant value to the multi-agent system in several ways:

1. **Enhanced Communication**: The Writer transforms technical or complex information into accessible, user-friendly content, improving the overall user experience.

2. **Coherent Integration**: By synthesizing information from multiple sources, the Writer creates a unified response that feels cohesive rather than disjointed.

3. **Balanced Presentation**: The Writer ensures that different perspectives and aspects of a topic receive appropriate attention, creating a more balanced final output.

4. **Improved Structure**: The Writer organizes information in a logical flow that guides the user through complex topics, making the content easier to understand and remember.

5. **Stylistic Consistency**: Despite drawing from multiple agents with potentially different writing styles, the Writer creates a consistent voice and tone throughout the response.

## Challenges and Considerations

While the Writer agent adds significant value to multi-agent systems, several challenges and considerations should be kept in mind:

1. **Information Fidelity**: The Writer must balance making content accessible with maintaining the accuracy of the original information. Simplifying too much can lead to inaccuracies or oversimplifications.

2. **Computational Overhead**: Adding a Writer agent increases the computational resources required for the system, as it involves an additional language model call.

3. **Conflicting Inputs**: The Writer may need to reconcile conflicting information or recommendations from the Researcher and Critic, requiring careful judgment.

4. **Audience Adaptation**: Different users may have different levels of expertise or preferences for how information is presented, which can be challenging for a one-size-fits-all Writer.

5. **Maintaining Attribution**: The Writer should maintain proper attribution of sources and clearly indicate when presenting different perspectives or contested information.

## Conclusion

The Writer agent represents a powerful addition to multi-agent AI systems, embodying the principle that effective communication requires not just accurate information, but also thoughtful organization and presentation. By synthesizing information from the Researcher and feedback from the Critic, the Writer helps ensure that the final output is not only accurate and comprehensive but also accessible and engaging.

In this notebook, we've explored:

1. The role and responsibilities of the Writer agent
2. How to implement a Writer agent using LangChain and GPT-4.1
3. How the Writer fits into a multi-agent workflow
4. The benefits and challenges of including a Writer agent
5. A visual representation of the information flow through the multi-agent system

As multi-agent systems continue to evolve, the role of the Writer is likely to become increasingly sophisticated, potentially incorporating personalization based on user preferences, adaptive communication styles for different audiences, and multimodal content generation that combines text with visual elements.